In [56]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import mlflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import git

In [57]:
# Загружаем один из файлов
data = np.load('../data/processed//Unit2_win50_str1_smp10.npz')

# Смотрим, какие массивы внутри
print(list(data.keys()))

['sample', 'label']


In [ ]:
# Указываем MLflow, куда отправлять данные
mlflow.set_tracking_uri("http://213.21.252.250:5000")

# Задаем имя эксперимента
mlflow.set_experiment("LSTM (test)")

# --- Получаем хеш коммита Git ---
try:
    repo = git.Repo(search_parent_directories=True)
    git_commit_hash = repo.head.object.hexsha
except Exception as e:
    git_commit_hash = "N/A" # На случай, если скрипт запущен не из Git-репозитория
    print(f"Warning: Could not get git commit hash. {e}")

print(f"Current Git Commit Hash: {git_commit_hash}")

# --- Параметры, которые нужно логировать ---
# Параметры из скрипта нарезки данных (sample_creator)
data_params = {
    "window_size": 100,
    "step": 2,
    "sampling_rate": 10
}

# Гиперпараметры модели
model_params = {
    "epochs": 2,
    "batch_size": 512,
    "validation_split": 0.2,
    "optimizer": "adam",
    "loss": "mean_squared_error"
}

Current Git Commit Hash: b4e962233791db132ece6527608aedef1f84670d


In [59]:
from gc import callbacks

from mlflow.keras.callback import MlflowCallback


with mlflow.start_run():
    print("Starting MLflow run...")

    # --- Логируем параметры ---
    mlflow.log_params(data_params)
    mlflow.log_params(model_params)
    mlflow.set_tag("git_commit", git_commit_hash)
    print("Parameters logged.")

    def load_and_merge_data(npz_units):
      sample_array_lst = []
      label_array_lst = []
      for npz_unit in npz_units:
        loaded = np.load(npz_unit)
        sample_array_lst.append(loaded['sample'])
        label_array_lst.append(loaded['label'])
      sample_array = np.dstack(sample_array_lst)
      label_array = np.concatenate(label_array_lst)
      sample_array = sample_array.transpose(2, 0, 1)
      return sample_array, label_array

    processed_dir = '../data/processed/'

    # Собираем пути к файлам для train и test
    train_files = [os.path.join(processed_dir, f) for f in os.listdir(processed_dir) if f.startswith(('Unit2_', 'Unit5_', 'Unit10_', 'Unit16_', 'Unit18_', 'Unit20_'))]
    test_files = [os.path.join(processed_dir, f) for f in os.listdir(processed_dir) if f.startswith(('Unit11_', 'Unit14_', 'Unit15_'))]
    print(train_files)

    # Загружаем данные
    X_train, y_train = load_and_merge_data(train_files)
    X_test, y_test = load_and_merge_data(test_files)

    print('Размер обучающей выборки (X):', X_train.shape)
    print('Размер обучающей выборки (y):', y_train.shape)
    print('Размер тестовой выборки (X):', X_test.shape)
    print('Размер тестовой выборки (y):', y_test.shape)

    # Определяем форму входных данных из X_train
    n_timesteps, n_features = X_train.shape[1], X_train.shape[2]

    # --- Создаем простую LSTM модель ---
    model = Sequential()
    model.add(LSTM(16, input_shape=(n_timesteps, n_features), return_sequences=True)) # return_sequences=True, если следующий слой тоже LSTM
    model.add(Dropout(0.2))
    model.add(LSTM(8))
    model.add(Dropout(0.2))
    model.add(Dense(1)) # Один выход, так как предсказываем одно число - RUL

    # Компилируем модель
    model.compile(optimizer=model_params['optimizer'], loss=model_params['loss'], metrics=['mae'])

    summary_list = []
    model.summary(print_fn=lambda x: summary_list.append(x))
    model_summary_string = "\n".join(summary_list)

    mlflow.log_text(model_summary_string, 'model_summary.txt')

    model.summary()

    # --- Обучаем модель ---
    history = model.fit(X_train, y_train, 
                        epochs=model_params['epochs'], 
                        batch_size=model_params['batch_size'], 
                        validation_split=model_params['validation_split'], # Используем часть данных для валидации на лету
                        callbacks=[mlflow.keras.MLflowCallback()],
                        verbose=1)

    # --- Оцениваем на тестовых данных ---
    loss, mae = model.evaluate(X_test, y_test, verbose=0)
    print(f'\nTest MAE: {mae:.2f}')

    metrics = {
        "mae": mae
    }
    mlflow.log_metrics(metrics)
    print(f"Metrics logged: {metrics}")

    # --- Логирование самой модели ---
    mlflow.keras.log_model(
        model,
        artifact_path="lstm-model", # Название папки с моделью в MLflow
    )
    print("Model logged as an artifact.")

    print("MLflow run finished successfully!")


Starting MLflow run...
Parameters logged.
['../data/processed/Unit16_win50_str1_smp10.npz', '../data/processed/Unit5_win50_str1_smp10.npz', '../data/processed/Unit18_win50_str1_smp10.npz', '../data/processed/Unit20_win50_str1_smp10.npz', '../data/processed/Unit2_win50_str1_smp10.npz', '../data/processed/Unit10_win50_str1_smp10.npz']
Размер обучающей выборки (X): (526051, 50, 20)
Размер обучающей выборки (y): (526051,)
Размер тестовой выборки (X): (125227, 50, 20)
Размер тестовой выборки (y): (125227,)


/Users/vladm/Projects/turbo-predict/.venv/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_22 (LSTM)                  │ (None, 50, 16)         │         2,368 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_22 (Dropout)            │ (None, 50, 16)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_23 (LSTM)                  │ (None, 8)              │           800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_23 (Dropout)            │ (None, 8)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 1)              │             9 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,177 (12.41 KB)

...

Epoch 1/2
822/822 ━━━━━━━━━━━━━━━━━━━━ 27s 31ms/step - loss: 1427.2161 - mae: 31.4132 - val_loss: 1335.5159 - val_mae: 29.4744
Epoch 2/2
822/822 ━━━━━━━━━━━━━━━━━━━━ 25s 31ms/step - loss: 1036.7698 - mae: 26.0769 - val_loss: 1030.2323 - val_mae: 26.1105


2025/11/16 04:24:59 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



Test MAE: 19.34
Metrics logged: {'mae': 19.33922004699707}


2025/11/16 04:25:00 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2025/11/16 04:25:06 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Model logged as an artifact.
MLflow run finished successfully!
🏃 View run whimsical-grouse-573 at: http://213.21.252.250:5000/#/experiments/1/runs/9e964ea12575438283cae5ca7e325440
🧪 View experiment at: http://213.21.252.250:5000/#/experiments/1
